# **🧝 ELFCASTL 🧝**

### **✨ Input Requirements ✨**
### ----------------------------

In [1]:
# Full Directory Path for Input Spectrum (WAVELENGTH, FLUX, NOISE)
input_file = '/Users/hunter_brooks8/Desktop/Accident.csv'

# Input outfile file name (DO NOT PUT DIRECTORY OR FILETYPE)
output_file = 'Accident'

# Set the number of walkers
walkers = 28

# Set the number of steps
steps = 2500

# Set the number of discards
discard = 500

### ----------------------------

#### Imports All Required Packages

In [2]:
# Imports all required packages 
import emcee
import corner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator

#### Loads in Observational Spectra

In [3]:
# Loads in the observed spectrum into SPLAT
observed = pd.read_csv(input_file)

# Gets the observed wavelength
observed_wave = observed.iloc[:, 0].tolist()

# Gets the observed fluxes
observed_flux = observed.iloc[:, 1].tolist()
observed_flux = [x / np.nanpercentile(observed_flux, 99.9) for x in observed_flux]
observed_flux = [x if x >= 0 else 2.2250738585072014e-30 for x in observed_flux]
observed_flux = np.nan_to_num(observed_flux, nan=2.2250738585072014e-30)

#### Records All Spectral Grid Points

In [4]:
# States all grid points
grid_teff = [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400]
grid_logg = [4, 4.25, 4.5, 4.75, 5, 5.25, 5.5]
grid_metal = [-1.0, -0.5, 0.0, 0.5, 1.0]
grid_co = [0.5, 1.0, 1.5, 2.5]
grid_kzz = [2, 4, 7, 8, 9]
total_grid = {'Teff': [], 'logg': [], 'metallicity': [], 'C/O': [], 'log(kzz)': [], 'wavelength': [], 'flux': []}

# Records all grid points and the wavelength 
for i in range(len(grid_teff)):
    for j in range(len(grid_logg)): 
        for p in range(len(grid_metal)): 
            for l in range(len(grid_co)): 
                for h in range(len(grid_kzz)): 
                    lower_metal = grid_metal[p]
                    if lower_metal == 0.0: 
                        lower_metal = -0.00
                    model_name = f'/Volumes/HUNTER/BONES/SPECTRA/MODELS/ELFOWL/elfowl24_t{grid_teff[i]:.0f}_g{grid_logg[j]:.2f}_z{lower_metal:.2f}_kzz{grid_kzz[h]:.2f}_co{grid_co[l]:.2f}_CUSTOM.txt'
                    data = pd.read_csv(model_name, delimiter='\t')
                    
                    wave = np.array(data['wave'])  # Convert to numpy array
                    flux = np.array(data['flux'])  # Convert to numpy arra
                    
                    f_interp = interp1d(wave, flux, kind='linear', fill_value="extrapolate")
                    resampled_flux = f_interp(observed_wave)
                    percentile_999 = np.nanmax(resampled_flux)
                    resampled_flux /= percentile_999
        
                    total_grid['Teff'].append(grid_teff[i])                # Add a new Teff value
                    total_grid['logg'].append(grid_logg[j])                # Add a new logg value
                    total_grid['metallicity'].append(grid_metal[p])        # Add a new metallicity value
                    total_grid['C/O'].append(grid_co[l])                   # Add a new C/O ratio value
                    total_grid['log(kzz)'].append(grid_kzz[h])             # Add a new log(kzz) value
                    total_grid['wavelength'].append(wave)                  # Overwrite the wavelength data
                    total_grid['flux'].append(resampled_flux)                        # Add flux data for the new set of parameters             


In [5]:
# Makes the total grid and fluxes and then makes an interpolation grid
point_grid = []
for i in range(len(total_grid['Teff'])): 
    point_grid.append([total_grid['Teff'][i], total_grid['logg'][i], total_grid['metallicity'][i], total_grid['C/O'][i], total_grid['log(kzz)'][i]])
point_grid = np.array(point_grid)
value_grid = np.array(total_grid['flux'])

#### Statistical Test Function AND Interpolation Between Grid Points

In [6]:
# Rounds to the nearest hundred
def closest_hundreds(value):
    lower_hundred = (value // 100) * 100
    upper_hundred = lower_hundred + 100
    return lower_hundred, upper_hundred

# Rounds to the nearest half
def closest_bounds(target, numbers):
    lower_bound = max(num for num in numbers if num <= target)
    upper_bound = min(num for num in numbers if num >= target)
    return lower_bound, upper_bound

# Defines the chi^2 statistical approach for our MCMC simulation
def rsquare(observed_wave, observed_flux, parm, total_grid): 
    # Loads in the model spectrum
    target_co = [0.5, 1.0, 1.5, 2.5]
    target_kzz = [2, 4, 7, 8, 9]
    target_logg = [4, 4.25, 4.5, 4.75, 5, 5.25, 5.5]
    target_metal = [-1.0, -0.5, 0.0, 0.5, 1.0]
    grid_parm = [
        *closest_hundreds(parm[0]),
        *closest_bounds(parm[1], target_logg),
        *closest_bounds(parm[2], target_metal),
        *closest_bounds(parm[3], target_co),
        *closest_bounds(parm[4], target_kzz)
    ]
    
    bounds = np.array([
        [int(grid_parm[0]), grid_parm[2], grid_parm[4], grid_parm[6], grid_parm[8]],
        [int(grid_parm[1]), grid_parm[3], grid_parm[5], grid_parm[7], grid_parm[9]]
    ])
    
    # Define grid points
    grid_points = np.array([
        [temp, logg, metal, co, kzz]
        for temp in bounds[:, 0]
        for logg in bounds[:, 1]
        for metal in bounds[:, 2]
        for co in bounds[:, 3]
        for kzz in bounds[:, 4]
    ])
    
    indices = np.array([np.where(np.all(point_grid == point, axis=1))[0][0] for point in grid_points])
    model_fluxes = value_grid[indices]
    
    model_flux = griddata(grid_points, model_fluxes, (parm[0], parm[1], parm[2], parm[3], parm[4]), method='linear', rescale=True)

    # Performs the chi square calculations and returns it
    observed_flux = np.array(observed_flux)
    model_flux = np.array(model_flux)
    
    # Normalize model flux to match the observed flux sum
    squared_differences = np.square(np.subtract(model_flux, observed_flux))
    mean_squared_difference = np.nanmean(squared_differences)
    rmse = np.sqrt(mean_squared_difference)
    
    return -10000*rmse

#### Makes the Parameter Space and Verifies the Walkers are Going in the Correct Direction

In [7]:
# Defines MCMC priors
def prior(parm):
    a, b, c, d, e, f, g = parm
    if 500 < a < 2400 and 4 < b < 5.5 and -1.0 < c < 0.5 and 0.5 < d < 2.5 and 2 < e < 9 and -0.2 < f < 0.2 and -0.001 < g < 0.001:
        return 0.0
    return -np.inf

# Defines the MCMC posteriors
def log_posterior(parm):
    lp = prior(parm)
    if not np.isfinite(lp):
        return -np.inf
    return lp + rsquare(observed_wave, observed_flux, parm, total_grid)

#### Set Up the MCMC Code Using the EMCEE Package (MAY TAKE A COUPLE OF MINUTES)

In [8]:
  # Runs the MCMC simulation #
# ------------------------------------------------------------ #
from pathos.multiprocessing import ProcessingPool as Pool

if __name__ == '__main__':
    ndim, nwalkers = 7, walkers
    n_steps = steps

    initial_positions = np.random.uniform(low=[1000, 5, -0.5, 0.5, 2, -0.05, -0.001], 
                                        high=[2000, 5.25, 0.5, 1.0, 8, 0.05, 0.001], 
                                        size=(nwalkers, ndim))

    with Pool() as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, pool=pool)
        sampler.run_mcmc(initial_positions, n_steps, progress=True)
# ------------------------------------------------------------ #

100%|██████████| 2500/2500 [05:33<00:00,  7.50it/s]


#### Finds the Best Values and the Errors From the Model

In [10]:
# Uses EMCEE's discard tool, finds the best parameters, and finds the uncertainties from the simulation
flat_samples = sampler.get_chain(discard=discard, thin=15, flat=True)
best_fit_params = []
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [5, 50, 95])
    best_fit_params.append([mcmc[0], mcmc[1], mcmc[2]])

# Prints these values
fraction_string = f"{'cm'}\u2044{'s^2'}"
chi2_final = -0.0001*rsquare(observed_wave, observed_flux, [best_fit_params[0][1], best_fit_params[1][1], best_fit_params[2][1], best_fit_params[3][1], best_fit_params[4][1], best_fit_params[5][1], best_fit_params[6][1]], total_grid)
print('#--------------------------------------------------------------------------------------------------------#')
print('                     Best Fit Parameters \u03C72 = ' + str(round((np.median(chi2_final)), 3)))
print('#--------------------------------------------------------------------------------------------------------#')
print("  Best Fit Parameters: Teff = " + str(round(best_fit_params[0][1], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][1], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][1], 3)) + ', C/O = ' + str(round(best_fit_params[3][1], 3)) + ', log(Kzz) = ' + str(round(best_fit_params[4][1], 3)))
print('#--------------------------------------------------------------------------------------------------------#')
print("    Lower Estimate: Teff = " + str(round(best_fit_params[0][0], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][0], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][0], 3)) + ', C/O = ' + str(round(best_fit_params[3][0], 3)) + ', log(Kzz) = ' + str(round(best_fit_params[4][0], 3)))
print("    Upper Estimate: Teff = " + str(round(best_fit_params[0][2], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][2], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][2], 3)) + ', C/O = ' + str(round(best_fit_params[3][2], 3)) + ', log(Kzz) = ' + str(round(best_fit_params[4][2], 3)))
print('#--------------------------------------------------------------------------------------------------------#')

#--------------------------------------------------------------------------------------------------------#
                     Best Fit Parameters χ2 = 0.236
#--------------------------------------------------------------------------------------------------------#
  Best Fit Parameters: Teff = 501.9K, log(g) = 5.474cm⁄s^2, [M/H] = -0.999, C/O = 0.607, log(Kzz) = 2.47
#--------------------------------------------------------------------------------------------------------#
    Lower Estimate: Teff = 500.2K, log(g) = 5.436cm⁄s^2, [M/H] = -1.0, C/O = 0.542, log(Kzz) = 2.028
    Upper Estimate: Teff = 507.2K, log(g) = 5.497cm⁄s^2, [M/H] = -0.995, C/O = 0.674, log(Kzz) = 3.879
#--------------------------------------------------------------------------------------------------------#


#### Plots the Comparison of the Spectrum, Walker Space, and a Corner Plot

In [11]:
# --------------------------------------------------------------------------------------------------------------------------------------------------- #
# Set plot font and create main figure with subplots
import os
plt.rcParams['font.family'] = 'Times New Roman'
fig, (ax_chain, ax_diff) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [2, 1]})

# Parameter chain plotting and CSV export
samples = sampler.get_chain()
labels = ["$T_{eff}$", "log(g)", "[M/H]", 'C/O', 'log(Kzz)', '∆F', '∆λ']
params_names = ['teff', 'logg', 'MH', 'CO', 'logKzz', 'delta_flux', 'delta_micron']

for i, (name, label) in enumerate(zip(params_names, labels)):
    flattened_array = samples[:, :, i].reshape((steps, walkers))
    df = pd.DataFrame(flattened_array, columns=[f'WALKER_{j+1}' for j in range(walkers)])
    df.to_csv(f'Output/{output_file}_parameter_{name}_ELFOWL.csv', index=False)

# Model flux calculation
target_co = [0.5, 1.0, 1.5, 2.5]
target_kzz = [2, 4, 7, 8, 9]
target_logg = [4, 4.25, 4.5, 4.75, 5, 5.25, 5.5]
target_metal = [-1.0, -0.5, 0.0, 0.5, 1.0]
grid_parm = [
    *closest_hundreds(best_fit_params[0][1]),
    *closest_bounds(best_fit_params[1][1], target_logg),
    *closest_bounds(best_fit_params[2][1], target_metal),
    *closest_bounds(best_fit_params[3][1], target_co),
    *closest_bounds(best_fit_params[4][1], target_kzz)
]

bounds = np.array([
    [int(grid_parm[0]), grid_parm[2], grid_parm[4], grid_parm[6], grid_parm[8]],
    [int(grid_parm[1]), grid_parm[3], grid_parm[5], grid_parm[7], grid_parm[9]]
])

# Define grid points
grid_points = np.array([
    [temp, logg, metal, co, kzz]
    for temp in bounds[:, 0]
    for logg in bounds[:, 1]
    for metal in bounds[:, 2]
    for co in bounds[:, 3]
    for kzz in bounds[:, 4]
])

indices = np.array([np.where(np.all(point_grid == point, axis=1))[0][0] for point in grid_points])
model_fluxes = value_grid[indices]

model_flux = griddata(grid_points, model_fluxes, (best_fit_params[0], best_fit_params[1][1], best_fit_params[2][1], best_fit_params[3][1], best_fit_params[4][1]), method='linear', rescale=True)

# Plot observed and model spectra
ax_chain.plot(observed_wave, observed_flux, lw=2, c='k', label='Observed')

flux_diff = np.subtract(observed_flux, model_flux[0])
ax_diff.axhline(0, c='k', lw=2)
ax_diff.plot(observed_wave, flux_diff, c='fuchsia')

model_flux = gaussian_filter1d(model_flux, 1.4)
ax_chain.plot(observed_wave, model_flux[0], lw=5, c='fuchsia', label='Best Fit Model', alpha=0.6)

# Save the fitted spectra
df = pd.DataFrame({'WAVELENGTH': observed_wave, 'FLUX': model_flux[0]})
df.to_csv(f'Output/{output_file}_fitted-spectra_ELFOWL.csv', index=False)

# Customize plot appearance
ax_chain.set_ylabel('Flux log$_{10}$(erg/s/cm$^{2}$/Å)', fontsize=25)
ax_diff.set_xlabel('Wavelength (µm)', fontsize=25)
ax_diff.set_ylabel('ΔFlux', fontsize=25)
for ax in [ax_chain, ax_diff]:
    ax.minorticks_on()
    ax.grid(True, alpha=0.3)
    ax.tick_params(which='minor', width=1, length=3, labelsize=10)
    ax.tick_params(which='major', width=2, length=6, labelsize=10)
ax_chain.tick_params(axis='x', labelsize=12)
ax_chain.tick_params(axis='y', labelsize=12, labelrotation=45)
ax_diff.tick_params(axis='x', labelsize=12)
ax_diff.tick_params(axis='y', labelsize=12, labelrotation=45)
ax_chain.legend(prop={'size': 20})
ax_chain.set_ylim(-0.1, 1.1)

plt.subplots_adjust(hspace=0)
plt.savefig('Output/figure1.png', dpi=200)
plt.close('all')
# --------------------------------------------------------------------------------------------------------------------------------------------------- #







# --------------------------------------------------------------------------------------------------------------------------------------------------- #
plt.rcParams['font.family'] = 'Times New Roman'
plt.figure(figsize=(5, 5))
upper_error_t, lower_error_t = round(best_fit_params[0][2] - best_fit_params[0][1], 1), round(best_fit_params[0][1] - best_fit_params[0][0], 1)
upper_error_g, lower_error_g = round(best_fit_params[1][2] - best_fit_params[1][1], 2), round(best_fit_params[1][1] - best_fit_params[1][0], 2)
upper_error_m, lower_error_m = round(best_fit_params[2][2] - best_fit_params[2][1], 2), round(best_fit_params[2][1] - best_fit_params[2][0], 2)
upper_error_c, lower_error_c = round(best_fit_params[3][2] - best_fit_params[3][1], 2), round(best_fit_params[3][1] - best_fit_params[3][0], 2)
upper_error_k, lower_error_k = round(best_fit_params[4][2] - best_fit_params[4][1], 2), round(best_fit_params[4][1] - best_fit_params[4][0], 2)

# Makes the limits for the corner plot
limits = [(best_fit_params[0][0] - (lower_error_t/2), best_fit_params[0][2] + (upper_error_t/2)), 
          (best_fit_params[1][0] - (lower_error_g/2), best_fit_params[1][2] + (upper_error_g/2)), 
          (best_fit_params[2][0] - (lower_error_m/2), best_fit_params[2][2] + (upper_error_m/2)), 
          (best_fit_params[3][0] - (lower_error_c/2), best_fit_params[3][2] + (upper_error_c/2)), 
          (best_fit_params[4][0] - (lower_error_k/2), best_fit_params[4][2] + (upper_error_k/2))]

# Define your filtered labels and truths
num_params_to_display = len(labels) - 2
filtered_labels = labels[:num_params_to_display]
filtered_truths = [best_fit_params[i][1] for i in range(num_params_to_display)]

# Create a mapping of labels to their index
label_to_index = {label: i for i, label in enumerate(labels)}

# Determine indices of the parameters to keep
indices_to_keep = [label_to_index[label] for label in filtered_labels]

# Filter the limits list to include only the relevant ranges
filtered_limits = [limits[i] for i in indices_to_keep]

# Filter flat_samples to include only relevant parameters
filtered_samples = np.array(flat_samples)[:, indices_to_keep]

# Create the corner plot
corner_fig = corner.corner(
    filtered_samples,
    labels=filtered_labels,
    truths=filtered_truths,
    truth_color='fuchsia',
    title_fmt='.2f',
    title_kwargs={'fontsize': 25},
    plot_contours=True,
    label_kwargs={'fontsize': 25},
    quantiles=[0.05, 0.5, 0.95],
    use_math_text=True,
    range=filtered_limits
)

# Sets the titles for the corner plots
titles = [fr'$Teff = {round(best_fit_params[0][1], 1)}^{{+{upper_error_t}}}_{{-{lower_error_t}}}$',
          fr'$log(g) = {round(best_fit_params[1][1], 2)}^{{+{upper_error_g}}}_{{-{lower_error_g}}}$', 
           fr'$[M/H] = {(round(best_fit_params[2][1], 2))}^{{+{upper_error_m}}}_{{-{lower_error_m}}}$', 
           fr'$C/O = {(round(best_fit_params[3][1], 2))}^{{+{upper_error_c}}}_{{-{lower_error_c}}}$', 
           fr'$log(Kzz) = {(round(best_fit_params[4][1], 2))}^{{+{upper_error_k}}}_{{-{lower_error_k}}}$']
list = [0, 6, 12, 18, 24]
for i in range(len(list)):
    ax = corner_fig.axes[list[i]]
    ax.set_title(titles[i], fontsize=12)
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.1)    
plt.savefig('Output/figure2.png', dpi = 200)
plt.close('all')
# --------------------------------------------------------------------------------------------------------------------------------------------------- #







# --------------------------------------------------------------------------------------------------------------------------------------------------- #
# Combine figures
fig, axs = plt.subplots(1, 2, figsize=(15, 8), gridspec_kw={'width_ratios': [1.25, 1]})

# Display scatter and corner plot images
scatter_img = plt.imread("Output/figure1.png")
axs[0].imshow(scatter_img)
axs[0].axis('off')
os.remove("Output/figure1.png")

corner_img = plt.imread("Output/figure2.png")
axs[1].imshow(corner_img)
axs[1].axis('off')
os.remove("Output/figure2.png")

plt.subplots_adjust(wspace=-0.5)
plt.tight_layout()
plt.savefig(f'Output/{output_file}_ELFOWL.pdf', dpi=500)
plt.close('all')
# --------------------------------------------------------------------------------------------------------------------------------------------------- #







# Plotting the results
# --------------------------------------------------------------------------------------------------------------------------------------------------- #
samples = sampler.get_chain()

# Create subplots
fig, axes = plt.subplots(ndim, figsize=(12, 8), sharex=True)

# Plot each parameter
for i in range(ndim):
    ax = axes[i]
    for j in range(nwalkers):
        ax.plot(samples[:, :, i].T[j], lw=1, color='k', alpha=0.5)
    ax.set_ylabel(f"Parameter {i+1}")
    ax.set_xlim(0, samples.shape[0])

# Label x-axis only for the last subplot
axes[-1].set_xlabel("Step number")

plt.tight_layout()
plt.savefig(f'Output/{output_file}_ELFOWL_STEPS.pdf')
plt.close('all')
# --------------------------------------------------------------------------------------------------------------------------------------------------- #

# **🎬 END OF ELFCASTL 🎬**